In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *
from numpy import *

In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
degree = UNITS.degree
N = UNITS.newton

<Unit('newton')>

In [3]:
default_params = Params(
    x = 0 * m,
    y = 10000 * m,
    speed = 88.889 * m/s,
    mass = 900 * kg,
    g = 3.7 * m/s**2,
    diam = 3 * m,
    C_d = 0.3,
    rho = 1.2 * 0.006 * kg/m**3,
    t_end = 3600 * s,
    angle = -45 * degree,
    Thrust = 11000 * kg * m/s**2
)

,values
x,0 meter
y,10000 meter
speed,88.889 meter / second
mass,900 kilogram
g,3.7 meter / second ** 2
diam,3 meter
C_d,0.3
rho,0.0072 kilogram / meter ** 3
t_end,3600 second
angle,-45 degree


In [4]:
def make_system(params):
    """ Make a system object.
    
    params: Parameter object includes
     - x: initial x position (m)
     - y: initial y position (m)
     - speed: initial speed (m/s)
     
     - mass: rover mass (kg)
     - g: acceleration due to gravity (m/s^2)
     
     - diam: rover diameter (m)
     - C_d: coefficient of drag (unitless)
     - rho: air density at surface level (kg/m^3)
     
     - t_end: end time (s)
     
     - angle: entry angle (degree)
     - Thrust: thrust force (N)
    
    returns: System object includes
     - mass: rover mass (kg)
     - area: rover surface area (m^2)
     - g: acceleration due to gravity (m/s^2)
     - thrust: acceleration due to thrust (m/s^2)
     - C_d: coefficient of drag (unitless)
     - rho: air density at surface level (kg/m^3)
     - t_end: end time (s)
     - init: State object includes
      - x
      - y
      - vx
      - vy
    """
    
    unpack(params)
    
    area = np.pi * (diam/2)**2
    
    thrust = Thrust / mass
    
    theta = np.deg2rad(angle)
    vx, vy = Vector(*pol2cart(theta, speed))
    init = State(x=x, y=y, vx=vx, vy=vy)
    
    return System(
        mass = mass,
        area = area,
        g = g,
        thrust = thrust,
        C_d = C_d,
        rho = rho,
        t_end = t_end,
        init = init
    )

In [5]:
make_system(default_params)

,values
mass,900 kilogram
area,7.0685834705770345 meter ** 2
g,3.7 meter / second ** 2
thrust,12.222222222222221 meter / second ** 2
C_d,0.3
rho,0.0072 kilogram / meter ** 3
t_end,3600 second
init,x 0 meter y ...


In [6]:
def drag_force(state, system):
    """Computes drag force in the opposite direction of velocity.
    
    state: State object with pos, v
    system: System object with area, C_d, rho
    
    returns: Vector drag force
    """
    
    x, y, vx, vy = state
    pos = Vector(x,y)
    v = Vector(vx,vy)
    unpack(system)
    
    C_a = np.exp(-(pos.y/1000)/np.e)
    mag = -(rho * C_a) * v.mag**2 * C_d * area / 2
    direction = v.hat()
    f_drag = mag * direction
    return f_drag

In [7]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, vx, vy)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (v.x, v.y, a.x, a.y)
    """
    x, y, vx, vy = state
    pos = Vector(x,y)
    v = Vector(vx,vy)
    unpack(system)
  
    a_drag = drag_force(state, system) / mass
    a_grav = Vector(0, -g)
    a_thrust = Vector(0, thrust)
    
    a = a_grav + a_thrust + a_drag
    
    return v.x, v.y, a.x, a.y

In [8]:
def event_func(state, t, system):
    """Stop when the y coordinate is 0.
    S
    state: State object
    t: time
    system: System object
    
    returns: y coordinate
    """
    x, y, vx, vy = state
    pos = Vector(x,y)
    v = Vector(vx,vy)
    
    return pos.y

In [9]:
def landing_speed(Thrust, params):
    """Computes the speed at which the rover hits the ground for a given thrust.
    
    Thrust: thrust force in kg * m/s^2
    params: Params object
    
    returns: landing speed in m/s
    """
    params = Params(params, Thrust=Thrust * kg * m/s**2)
    system = make_system(params)
    results, details = run_ode_solver(system, slope_func, events=event_func)
    final_v = Vector(get_last_value(results.vx), get_last_value(results.vy))
    final_speed = final_v.mag * m/s
    return final_speed

In [10]:
def thrust_for_angle(angle, params):
    """Computes the optimal thrust for a given angle.
    
    angle: entry angle in degrees
    params: Params object
    
    returns: thrust in kg * m/s^2
    """
    new_params = Params(params, angle=angle * degree)
    result = min_bounded(landing_speed, [0,10000], new_params)
    optimal_thrust = result.x
    return optimal_thrust

In [11]:
def safe_angle_ranges(threshold, params):
    """Computes all ranges of entry angles that can yield a safe landing speed.
    
    threshold: maximum safe landing speed in m/s
    params: Params object
    
    returns: list of tuples containing range boundaries
    """
    safe_angles = []
    
    for angle in range(-90, 1):
        thrust = thrust_for_angle(angle, params)
        params_w_angle = Params(params, angle = angle * degree)
        impact_speed = landing_speed(thrust, params_w_angle)
        print(f'{angle}: {impact_speed}')
        if impact_speed <= threshold:
            safe_angles.append(angle)
    
    return safe_angles

In [12]:
safe_angle_ranges(10 * m/s, default_params)

-90: 26.615912222140757 meter / second
-89: 36.28747429346087 meter / second
-88: 37.09117802221576 meter / second
-87: 31.148750560786578 meter / second
-86: 24.756356692362612 meter / second
-85: 10.932019650394478 meter / second
-84: 12.20636016530011 meter / second
-83: 26.38799765521477 meter / second
-82: 40.14103549865789 meter / second
-81: 38.15641399482601 meter / second
-80: 38.05928474437089 meter / second
-79: 38.00217898921286 meter / second
-78: 38.603050518105455 meter / second
-77: 39.196948450065 meter / second
-76: 39.700275712808484 meter / second
-75: 40.07081341035919 meter / second
-74: 40.35746261209597 meter / second
-73: 40.63594865027769 meter / second
-72: 40.96053101599401 meter / second
-71: 43.496692422747685 meter / second
-70: 43.513421284004906 meter / second
-69: 43.61622724548074 meter / second
-68: 43.80113639685913 meter / second
-67: 44.06315590769954 meter / second
-66: 44.39686325707829 meter / second
-65: 44.796542733782054 meter / second
-64: 

C:\Miniconda3\lib\site-packages\pint\quantity.py:715: RuntimeWarning: overflow encountered in double_scalars
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


-9: 46.024544025855974 meter / second
-8: 40.43290528997372 meter / second
-7: 36.336337767837826 meter / second
-6: 34.33667673189016 meter / second
-5: 34.13692761383503 meter / second
-4: 36.66701270350553 meter / second
-3: 42.69299727166651 meter / second
-2: 48.9387378604693 meter / second
-1: 54.83312018528552 meter / second
0: 60.01617520723515 meter / second


[]